<a href="https://colab.research.google.com/github/isabellasims/Google-colabs/blob/master/geopandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install geopandas
import geopandas as gpd #rename as gpd
import math

In [ ]:
import folium # docs: https://python-visualization.github.io/folium/quickstart.html#Getting-Started
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10, user_agent = "dlab.berkeley.edu-workshop")

In [ ]:
# Load the data
# data from http://crimemap.dc.gov/Download.aspx
crimes = pd.read_csv("/content/SearchResults (2).txt", encoding='latin-1')



# get address in order to geocode lat long coords for markers
crimes['STREET'] = crimes['BLOCK'].str.split('OF').str[1] + ', WASHINGTON DC'


# Grab relevant columns
columnsTitles = ['REPORT_DAT', 'OFFENSE', 'METHOD','STREET','LAT','LONG','LOC','WARD','DISTRICT','SHIFT','NEIGHBORHOOD_CLUSTER','VOTING_PRECINCT']

crimes = crimes.reindex(columns=columnsTitles)
#crimes['LAT'] = geolocator.geocode(crimes['STREET']).latitude


location = geolocator.geocode(crimes['STREET'][1]).latitude
# location.latitude
# location.longitude
location




38.9117275

In [156]:
homicides = crimes[crimes.OFFENSE == 'HOMICIDE'] # create homicide df

# add geocoded location, lat, and long points to df
def addLoc(street):
    return geolocator.geocode(street)

def addLat(loc):
    return geolocator.geocode(loc).latitude

def addLong(loc):
  return geolocator.geocode(loc).longitude

homicides['LOC'] = homicides['STREET'].apply(addLoc)
homicides['LAT'] = homicides['LOC'].apply(addLat)
homicides['LONG'] = homicides['LOC'].apply(addLong)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [ ]:
homicides.head()

In [ ]:
# Create a map
m_2 = folium.Map(location=[38.9072,-77.0369], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in homicides.iterrows():
    Marker([row['LAT'], row['LONG']]).add_to(m_2)

# Display the map
m_2

In [160]:
m_3 = folium.Map(location=[38.9072,-77.0369], tiles='cartodbpositron', zoom_start=13)

nightTimeHomicides = homicides[homicides.SHIFT == 'MIDNIGHT']


# Add points to the map
mc = MarkerCluster()
for idx, row in nightTimeHomicides.iterrows():
    if not math.isnan(row['LONG']) and not math.isnan(row['LAT']):
        mc.add_child(Marker([row['LAT'], row['LONG']]))
m_3.add_child(mc)

# Display the map
m_3